# 层和块
## 自定义块

In [2]:
import torch
from torch import nn
from torch.nn import functional as F

In [5]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.out = nn.Linear(256, 10)
    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))

实例化多层感知机的层，然后在每次调用正向传播函数的时候调用这些层

In [7]:
X = torch.rand(2, 20)
net = MLP()
net(X)

tensor([[ 0.1022, -0.1942, -0.1102, -0.0658,  0.0759, -0.1352, -0.0582,  0.1205,
         -0.0920,  0.1517],
        [ 0.0836, -0.2122, -0.1163, -0.0720,  0.0697, -0.0948, -0.0827,  0.1174,
         -0.0299,  0.0972]], grad_fn=<AddmmBackward0>)

## 顺序块

In [20]:
# 自定义的nn.Sequential
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for block in args:
            self._modules[block] = block
    
    def forward(self, X):
        for block in self._modules.values():
            X = block(X)
        return X

In [21]:
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[ 0.1915, -0.1373, -0.0960,  0.0674, -0.0280, -0.0877, -0.0079,  0.2803,
         -0.0391,  0.0705],
        [ 0.2589, -0.1264, -0.1165,  0.0372,  0.0298, -0.0565, -0.0856,  0.3052,
          0.0934,  0.1327]], grad_fn=<AddmmBackward0>)

## 在正向传播函数中执行代码

In [24]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.rand_weight = torch.rand((20, 20), requires_grad=False) # 不参加训练，无梯度
        self.linear = nn.Linear(20, 20)
    
    def forward(self, X):
        X = self.linear(X)
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        X = self.linear(X)
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

In [25]:
net = FixedHiddenMLP()
net(X)

tensor(-0.0266, grad_fn=<SumBackward0>)

## 混合搭配各种组合块

In [28]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)
        
    def forward(self, X):
        return self.linear(self.net(X))

In [29]:
chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(-0.1964, grad_fn=<SumBackward0>)